exploratoryanalysis.R
Jay Sayre - sayrejay (at) gmai|,

NOTE: cleanuptextable.py MUST BE RUN AFTER RUNNING THIS SCRIPT TO UPDATE TABLES IN PAPER

INPUTS: 
"municipality_level_DATASET.csv" - As titled, produced by dataassembly.py

"mun_level_isic4dig_DATASET.csv" - Same as above, except uses ISIC 4 digit and only empresas data (not IPUMS), produced by dataassembly.py

"municipality_occupation_level_DATASET.csv" - As titled, produced by dataassembly.py

OUTPUTS:

"../Plots/munregs.tex" - Municipality level results

"../Plots/mun4digregs.tex" - Municipality level results (ISIC 4 digit/no IPUMS occupation)

"../Plots/munoccregs.tex" - Occupation level results

In [1]:
library(ggplot2)
library(texreg)
library(tikzDevice)

## Directory information
setwd("~/Dropbox/College/DR_Paper/cafta-dr/Output/")
#setwd("D:/Dropbox/Dropbox (Personal)/College/DR_Paper/")
plotdir <- "../Plots/"

## INPUTS 
mundf <- read.csv("municipality_level_DATASET.csv")
munisic4df <- read.csv("mun_level_isic4dig_DATASET.csv")
munoccdf <- read.csv("municipality_occupation_level_DATASET.csv")

## OUTPUTS 
munlevelresults <- paste(plotdir,"munregs.tex",sep="")
mun4diglevelresults <- paste(plotdir,"mun4digregs.tex",sep="")
occupationlevelresults <- paste(plotdir,"munoccregs.tex",sep="")

Version:  1.36
Date:     2015-12-08
Author:   Philip Leifeld (Eawag & University of Bern)

Please cite the JSS article in your publications -- see citation("texreg").


In [2]:
### For clustered standard errors
### http://www.r-bloggers.com/texreg-a-package-for-beautiful-and-easily-customizable-latex-regression-tables-from-r/
                    
robust.se <- function(model, cluster){
  require(sandwich)
  M <- length(unique(cluster))
  N <- length(cluster)
  K <- model$rank
  dfc <- (M/(M - 1)) * ((N - 1)/(N - K))
  uj <- apply(estfun(model), 2, function(x) tapply(x, cluster, sum));
  rcse.cov <- dfc * sandwich(model, meat = crossprod(uj)/N)
  rcse.se <- coeftest(model, rcse.cov)
  return(list(rcse.cov, rcse.se))
}
 
m3 <- robust.se(m2,dat$g)[[2]]
                        
coeftest <- function(x, vcov. = NULL, df = NULL, ...)
{
  UseMethod("coeftest")
}

coeftest.default <- function(x, vcov. = NULL, df = NULL, ...)
{
  ## use S4 methods if loaded
  coef0 <- if("stats4" %in% loadedNamespaces()) stats4::coef else coef
  vcov0 <- if("stats4" %in% loadedNamespaces()) stats4::vcov else vcov

  ## extract coefficients and standard errors
  est <- coef0(x)
  if(is.null(vcov.)) se <- vcov0(x) else {
      if(is.function(vcov.)) se <- vcov.(x)
        else se <- vcov.
  }
  se <- sqrt(diag(se))

  ## match using names and compute t/z statistics
  if(!is.null(names(est)) && !is.null(names(se))) {
    if(length(unique(names(est))) == length(names(est)) && length(unique(names(se))) == length(names(se))) {
      anames <- names(est)[names(est) %in% names(se)]
      est <- est[anames]
      se <- se[anames]
    }
  }  
  tval <- as.vector(est)/se

  ## apply central limit theorem
  if(is.null(df)) {
    df <- try(df.residual(x), silent = TRUE)
    if(inherits(df, "try-error")) df <- NULL
  }
  if(is.null(df)) df <- 0

  if(is.finite(df) && df > 0) {
    pval <- 2 * pt(abs(tval), df = df, lower.tail = FALSE)
    cnames <- c("Estimate", "Std. Error", "t value", "Pr(>|t|)")
    mthd <- "t"
  } else {
    pval <- 2 * pnorm(abs(tval), lower.tail = FALSE)
    cnames <- c("Estimate", "Std. Error", "z value", "Pr(>|z|)")
    mthd <- "z"
  }
  rval <- cbind(est, se, tval, pval)
  colnames(rval) <- cnames
  class(rval) <- "coeftest"
  attr(rval, "method") <- paste(mthd, "test of coefficients")
  ##  dQuote(class(x)[1]), "object", sQuote(deparse(substitute(x))))
  return(rval)
} 

coeftest.glm <- function(x, vcov. = NULL, df = Inf, ...)
  coeftest.default(x, vcov. = vcov., df = df, ...)  

coeftest.mlm <- function(x, vcov. = NULL, df = NULL, ...)
{
  ## obtain vcov
  v <- if(is.null(vcov.)) vcov(x) else if(is.function(vcov.)) vcov.(x) else vcov.

  ## nasty hack: replace coefficients so that their names match the vcov() method
  x$coefficients <- structure(as.vector(x$coefficients), .Names = colnames(vcov(x)))

  ## call default method
  coeftest.default(x, vcov. = v, df = df, ...)
}

coeftest.survreg <- function(x, vcov. = NULL, df = Inf, ...)
{
  if(is.null(vcov.)) v <- vcov(x) else {
      if(is.function(vcov.)) v <- vcov.(x)
  	else v <- vcov.
  }
  if(length(x$coefficients) < NROW(x$var)) {
    x$coefficients <- c(x$coefficients, "Log(scale)" = log(x$scale))
  }
  coeftest.default(x, vcov. = v, df = df, ...)  
} 

coeftest.breakpointsfull <- function(x, vcov. = NULL, df = NULL, ...)
{
  est <- coef(x, ...)
  if(is.null(df)) {
    df <- df.residual(x, ...)
    df <- as.vector(rep(df, rep(NCOL(est), length(df))))
  }  

  rnames <- as.vector(t(outer(rownames(est), colnames(est), paste)))
  est <- as.vector(t(est))
  
  se <- vcov(x, vcov. = vcov., ...)

  se <- as.vector(sapply(seq(along = se), function(x) sqrt(diag(se[[x]]))))
  tval <- est/se

  if(any(is.finite(df) && df > 0)) {
    pval <- 2 * pt(abs(tval), df = df, lower.tail = FALSE)
    cnames <- c("Estimate", "Std. Error", "t value", "Pr(>|t|)")
    mthd <- "t"
  } else {
    pval <- 2 * pnorm(abs(tval), lower.tail = FALSE)
    cnames <- c("Estimate", "Std. Error", "z value", "Pr(>|z|)")
    mthd <- "z"
  }
  rval <- cbind(est, se, tval, pval)
  colnames(rval) <- cnames
  rownames(rval) <- rnames
  class(rval) <- "coeftest"
  attr(rval, "method") <- paste(mthd, "test of coefficients")
  ##  dQuote(class(x)[1]), "object", sQuote(deparse(substitute(x))))
  return(rval)
} 

print.coeftest <- function(x, ...)
{
  mthd <- attr(x, "method")
  if(is.null(mthd)) mthd <- "Test of coefficients"
  cat(paste("\n", mthd,":\n\n", sep = ""))
  printCoefmat(x, ...)
  cat("\n")
  invisible(x)
}

Loading required package: sandwich


ERROR: Error in unique(cluster): object 'dat' not found


In [3]:
### Cleanup variables in municipality level dataset
mundf$chnginc1 <- mundf$grossalary13-mundf$inc02
mundf$chnginc2 <- mundf$frstsourcinc13-mundf$inc02
mundf$chnginc3 <- mundf$occinc13-mundf$inc02

mundf$prov <- factor(mundf$prov)
mundf$chngtrf <- mundf$duty02-mundf$duty13

In [4]:
### Cleanup variables in municipality level (ISIC 4) dataset
munisic4df$chnginc1 <- munisic4df$occinc13-munisic4df$inc02
munisic4df$chnginc2 <- log(1+munisic4df$grossalary13)-log(1+munisic4df$inc02)
munisic4df$chnginc3 <- munisic4df$frstsourcinc13-munisic4df$inc02
munisic4df$prov <- factor(munisic4df$prov)
munisic4df$chngtrf <- munisic4df$duty13-munisic4df$duty02

In [5]:
### Cleanup variables in municipality/occupation level dataset
munoccdf$chnginc1 <- munoccdf$frstsourcinc13-munoccdf$inc2002
munoccdf$chnginc2 <- munoccdf$grossalary13-munoccdf$inc2002
munoccdf$chnginc3 <- munoccdf$occinc13-munoccdf$inc2002
munoccdf$chngtrf <- munoccdf$duty02-munoccdf$duty13
munoccdf$lgchng <- log(munoccdf$chngtrf+1)
munoccdf$lgdty02 <-log(munoccdf$duty02+1)
munoccdf$chngwrk <- munoccdf$numworkers10-munoccdf$numworkers02
munoccdf$occ <- factor(munoccdf$occ)
munoccdf$munocc <- factor(munoccdf$munocc)

## Make a different dataset with only occupations producing tradables
tradedsectordf <- subset(munoccdf, nontraded == 0) 

In [6]:
### Municipality level regressions
#colnames(mundf)
munreg1 <- lm(chnginc1~chngtrf, data = mundf)
munreg2 <- lm(chnginc1~chngtrf+prov, data = mundf)
robmunreg1 <- robust.se(munreg1, mundf$prov)[[2]]
robmunreg2 <- robust.se(munreg2, mundf$prov)[[2]]

#summary(munreg1)
#plot(mundf$chnginc3,mundf$chngtrf)

texreg(list(munreg1,munreg2),#,munreg3),
        file=munlevelresults,
        stars = c(0.01, 0.05,0.10),
        caption="Municipality level regressions",
        caption.above = TRUE, 
        override.se=list(robmunreg1[,2],robmunreg2[,2]),
        override.pval=list(robmunreg1[,4],robmunreg2[,4]),
        omit.coef="prov",
        custom.model.names=c("(1)","(2)"))#,"(3)"))


The table was written to the file '../Plots/munregs.tex'.



In [7]:
### Municipality level (ISIC 4 digit) regressions
#colnames(mundf)
mun4reg1 <- lm(chnginc2~chngtrf, data = munisic4df)
mun4reg2 <- lm(chnginc2~chngtrf+prov, data = munisic4df)
mun4reg3 <- lm(chnginc3~chngtrf+prov, data = munisic4df)
robmun4reg1 <- robust.se(mun4reg1, munisic4df$prov)[[2]]
robmun4reg2 <- robust.se(mun4reg2, munisic4df$prov)[[2]]
robmun4reg3 <- robust.se(mun4reg3, munisic4df$prov)[[2]]

#summary(mun4reg1)

texreg(list(mun4reg1,mun4reg2,mun4reg3),
        file=mun4diglevelresults,
        stars = c(0.01, 0.05,0.10),
        caption="Municipality level regressions",
        caption.above = TRUE, 
        override.se=list(robmun4reg1[,2],robmun4reg2[,2],robmun4reg3[,2]),
        override.pval=list(robmun4reg1[,4],robmun4reg2[,4],robmun4reg3[,4]),
        omit.coef="prov",
        custom.model.names=c("(1)","(2)","(3)"))

The table was written to the file '../Plots/mun4digregs.tex'.



In [8]:
### Municipality/occupation level regressions
#colnames(munoccdf)
reg1 <- lm(chnginc2~chngtrf+munocc,data = tradedsectordf)
reg2 <- lm(chnginc2~chngtrf+munocc,data = munoccdf)
reg3 <- lm(chnginc2~chngtrf+chngwrk,data = munoccdf)
reg4 <- lm(chnginc2~chngtrf+chngwrk+munocc,data = munoccdf)
robreg1 <- robust.se(reg1, tradedsectordf$munocc)[[2]]
robreg2 <- robust.se(reg2, munoccdf$munocc)[[2]]
robreg3 <- robust.se(reg3, munoccdf$munocc)[[2]]
robreg4 <- robust.se(reg4, munoccdf$munocc)[[2]]

#summary(reg1)
#plot(munoccdf$chnginc2,munoccdf$chngtrf)

texreg(list(reg1,reg2,reg3,reg4),
        file=occupationlevelresults,
        stars = c(0.01, 0.05,0.10),
        caption="Municipality/occupation level regressions",
        caption.above = TRUE,
        override.se=list(robreg1[,2],robreg2[,2],robreg3[,2],
                        robreg4[,2]),
        override.pval=list(robreg1[,4],robreg2[,4],robreg3[,4],
                        robreg4[,4]),
        omit.coef="munocc|z",
        custom.model.names=c("(1)","(2)","(3)","(4)"))

The table was written to the file '../Plots/munoccregs.tex'.



In [12]:
reg1 <- lm(chnginc1~chngtrf+munocc,data = tradedsectordf)
reg2 <- lm(chnginc1~chngtrf+munocc,data = munoccdf)

summary(reg1)
summary(reg2)


Call:
lm(formula = chnginc1 ~ chngtrf + munocc, data = tradedsectordf)

Residuals:
   Min     1Q Median     3Q    Max 
-37523  -1655    115   2153 134465 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -6468.71    3472.47  -1.863  0.06310 .  
chngtrf       309.15     114.95   2.689  0.00741 ** 
munocc201     -29.23    4308.08  -0.007  0.99459    
munocc202    4805.61    6811.99   0.705  0.48087    
munocc203    2137.08    6140.19   0.348  0.72796    
munocc204    3344.85    6813.87   0.491  0.62373    
munocc205    3332.89    7999.90   0.417  0.67715    
munocc206    5804.97    5391.62   1.077  0.28218    
munocc208    2455.88    6813.87   0.360  0.71869    
munocc209     727.81    7999.90   0.091  0.92755    
munocc301    8597.29    5701.36   1.508  0.13224    
munocc302    3005.87    5699.04   0.527  0.59814    
munocc303    2899.30    5386.77   0.538  0.59067    
munocc304    1723.14   10779.43   0.160  0.87306    
munocc305    1989.12   10779.43   


Call:
lm(formula = chnginc1 ~ chngtrf + munocc, data = munoccdf)

Residuals:
   Min     1Q Median     3Q    Max 
-37426  -1361    234   1890 136874 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -4258.89    2831.31  -1.504  0.13304    
chngtrf       112.17      94.24   1.190  0.23440    
munocc201    -782.71    3778.44  -0.207  0.83596    
munocc202    3676.52    5782.88   0.636  0.52517    
munocc203    1562.78    5323.23   0.294  0.76918    
munocc204    2954.86    5784.56   0.511  0.60966    
munocc205    3067.69    6482.23   0.473  0.63620    
munocc206    4074.26    4546.69   0.896  0.37055    
munocc208    1706.88    6485.75   0.263  0.79250    
munocc209     446.85    7700.95   0.058  0.95375    
munocc301    6643.80    4742.73   1.401  0.16177    
munocc302    1922.16    5324.55   0.361  0.71822    
munocc303    2225.06    4545.08   0.490  0.62462    
munocc304    2827.67    7682.48   0.368  0.71295    
munocc305    1708.15   10509.55   0.163 